# HE Team LLM Assistant - API Examples

This notebook demonstrates how to use all the API endpoints of the HE Team LLM Assistant.

**Base URL**: `http://10.252.38.241:8000`

**API Version**: v2.1.1 (2025-10-17)

## Table of Contents
1. [Setup & Authentication](#setup)
2. [User Management](#user-management)
3. [Chat & Conversations](#chat)
4. [Conversation History](#conversation-history)
5. [Web Search](#web-search)
6. [RAG (Knowledge Base)](#rag)
7. [File Management](#file-management)
   - 7.5 [JSON Processing](#json-processing)
8. [System Health & Configuration](#system)

## 1. Setup & Authentication <a name="setup"></a>

In [ ]:
import requests
import json
from typing import Dict, Optional
from pprint import pprint

# Base URL for all API calls
BASE_URL = "http://10.252.38.241:8000"

# Global variable to store session token
session_token = None

def get_headers(token: Optional[str] = None) -> Dict[str, str]:
    """Generate headers for API requests"""
    headers = {"Content-Type": "application/json"}
    if token:
        headers["Authorization"] = f"Bearer {token}"
    return headers

def print_response(response: requests.Response, title: str = ""):
    """Pretty print API response"""
    if title:
        print(f"\n{'='*60}")
        print(f"{title}")
        print(f"{'='*60}")
    print(f"Status Code: {response.status_code}")
    try:
        print("Response:")
        pprint(response.json(), width=100)
    except:
        print(response.text)
    print()

### 1.1 Login

In [ ]:
# Login with default admin credentials
login_data = {
    "username": "admin",
    "password": "administrator"
}

response = requests.post(
    f"{BASE_URL}/api/auth/login",
    json=login_data,
    headers=get_headers()
)

print_response(response, "LOGIN")

if response.status_code == 200:
    session_token = response.json()["session_token"]
    print(f"Session token stored: {session_token[:20]}...")
else:
    print("Login failed!")

### 1.2 Get Current User Info

In [ ]:
# Get current authenticated user information
response = requests.get(
    f"{BASE_URL}/api/auth/me",
    headers=get_headers(session_token)
)

print_response(response, "CURRENT USER INFO")

### 1.3 System Health Check (No Auth Required)

In [ ]:
# Check system health
response = requests.get(f"{BASE_URL}/health")

print_response(response, "SYSTEM HEALTH")

## 2. User Management <a name="user-management"></a>

Admin-only operations for managing users.

### 2.1 Create New User

In [ ]:
# Create a new user (admin only)
new_user_data = {
    "username": "testuser",
    "password": "testpassword123",
    "email": "testuser@heteam.com",
    "role": "user",
    "display_name": "Test User"
}

response = requests.post(
    f"{BASE_URL}/api/admin/users",
    json=new_user_data,
    headers=get_headers(session_token)
)

print_response(response, "CREATE USER")

### 2.2 List All Users

In [ ]:
# List all users (admin only)
response = requests.get(
    f"{BASE_URL}/api/admin/users",
    headers=get_headers(session_token)
)

print_response(response, "LIST USERS")

### 2.3 Update User

In [ ]:
# Update user details (admin only)
update_data = {
    "email": "newemail@heteam.com",
    "display_name": "Updated Test User"
}

response = requests.put(
    f"{BASE_URL}/api/admin/users/testuser",
    json=update_data,
    headers=get_headers(session_token)
)

print_response(response, "UPDATE USER")

## 3. Chat & Conversations <a name="chat"></a>

### 3.1 Send a Simple Chat Message

In [ ]:
# Send a chat message
chat_data = {
    "message": "What is machine learning?",
    "temperature": 0.7,
    "max_tokens": 2000
}

response = requests.post(
    f"{BASE_URL}/api/chat/messages",
    json=chat_data,
    headers=get_headers(session_token)
)

print_response(response, "CHAT MESSAGE")

# Store session_id for continuing the conversation
if response.status_code == 200:
    chat_session_id = response.json().get("session_id")
    print(f"Session ID: {chat_session_id}")

### 3.2 Continue the Conversation

In [ ]:
# Continue the conversation using the same session_id
follow_up_data = {
    "message": "Can you give me an example of a machine learning algorithm?",
    "session_id": chat_session_id  # Use the session_id from previous chat
}

response = requests.post(
    f"{BASE_URL}/api/chat/messages",
    json=follow_up_data,
    headers=get_headers(session_token)
)

print_response(response, "FOLLOW-UP MESSAGE")

### 3.3 Chat with Web Search

In [ ]:
# Send a chat message with web search enabled
web_search_data = {
    "message": "What are the latest AI developments in 2025?"
}

response = requests.post(
    f"{BASE_URL}/api/chat/web-search",
    json=web_search_data,
    headers=get_headers(session_token)
)

print_response(response, "WEB SEARCH CHAT")

## 4. Conversation History <a name="conversation-history"></a>

### 4.1 List All Conversations

In [ ]:
# Get all conversation sessions for current user
response = requests.get(
    f"{BASE_URL}/api/chat/sessions",
    headers=get_headers(session_token)
)

print_response(response, "ALL CONVERSATIONS")

# Store the first session_id for demonstration
if response.status_code == 200 and response.json().get("sessions"):
    demo_session_id = response.json()["sessions"][0]["id"]
    print(f"Using session ID for examples: {demo_session_id}")

### 4.2 Get Specific Conversation History

In [ ]:
# Get history for a specific session
response = requests.get(
    f"{BASE_URL}/api/chat/sessions/{chat_session_id}",
    headers=get_headers(session_token)
)

print_response(response, "CONVERSATION HISTORY")

### 4.3 Get Conversation Metadata

In [ ]:
# Get conversation metadata
response = requests.get(
    f"{BASE_URL}/api/conversations/{chat_session_id}",
    headers=get_headers(session_token)
)

print_response(response, "CONVERSATION METADATA")

## 5. Web Search <a name="web-search"></a>

### 5.1 Perform Web Search

In [ ]:
# Perform a web search
search_data = {
    "query": "machine learning algorithms",
    "max_results": 5
}

response = requests.post(
    f"{BASE_URL}/api/search/web",
    json=search_data,
    headers=get_headers(session_token)
)

print_response(response, "WEB SEARCH")

### 5.2 Extract Keywords from Text

In [ ]:
# Extract keywords using LLM
keyword_data = {
    "text": "I want to learn about machine learning algorithms and neural networks for image recognition"
}

response = requests.post(
    f"{BASE_URL}/api/search/extract-keywords",
    json=keyword_data,
    headers=get_headers(session_token)
)

print_response(response, "KEYWORD EXTRACTION")

### 5.3 Get Search System Status

In [ ]:
# Get search system capabilities
response = requests.get(
    f"{BASE_URL}/api/search/status",
    headers=get_headers(session_token)
)

print_response(response, "SEARCH SYSTEM STATUS")

## 6. RAG (Knowledge Base) <a name="rag"></a>

RAG allows you to query a knowledge base built from uploaded JSON files.

### 6.1 Get RAG System Statistics

In [ ]:
# Get RAG system statistics
response = requests.get(
    f"{BASE_URL}/api/rag/stats",
    headers=get_headers(session_token)
)

print_response(response, "RAG STATISTICS")

### 6.2 Search the Knowledge Base

In [ ]:
# Search the knowledge base
rag_search_data = {
    "query": "What are users saying about battery life?",
    "n_results": 5
}

response = requests.post(
    f"{BASE_URL}/api/rag/search",
    json=rag_search_data,
    headers=get_headers(session_token)
)

print_response(response, "RAG SEARCH")

### 6.3 Chat with RAG Context

In [ ]:
# Send a chat message with RAG enabled
rag_chat_data = {
    "message": "What do the documents say about Fold 7 battery performance?",
    "use_rag": True  # Enable RAG context injection
}

response = requests.post(
    f"{BASE_URL}/api/chat/messages",
    json=rag_chat_data,
    headers=get_headers(session_token)
)

print_response(response, "RAG-ENABLED CHAT")

### 6.4 Get RAG Context for a Query

In [ ]:
# Get formatted context from RAG system
response = requests.get(
    f"{BASE_URL}/api/rag/context",
    params={"query": "battery life", "max_length": 1000},
    headers=get_headers(session_token)
)

print_response(response, "RAG CONTEXT")

## 7. File Management <a name="file-management"></a>

Upload files for analysis and query them with AI.

### 7.1 Upload a File

In [ ]:
# Upload a file
# Note: You need to have a file to upload. This example uses a text file.

# First, create a sample file
sample_content = """This is a sample document about machine learning.
Machine learning is a method of data analysis that automates analytical model building.
It is a branch of artificial intelligence based on the idea that systems can learn from data.
"""

with open("sample_document.txt", "w") as f:
    f.write(sample_content)

# Upload the file
with open("sample_document.txt", "rb") as f:
    files = {"file": ("sample_document.txt", f, "text/plain")}
    response = requests.post(
        f"{BASE_URL}/api/files/upload",
        files=files,
        headers={"Authorization": f"Bearer {session_token}"}  # Don't include Content-Type for multipart
    )

print_response(response, "FILE UPLOAD")

# Store file_id for later use
if response.status_code == 201:
    uploaded_file_id = response.json().get("file_id")
    print(f"Uploaded file ID: {uploaded_file_id}")

### 7.2 List All Uploaded Files

In [ ]:
# List all uploaded files
response = requests.get(
    f"{BASE_URL}/api/files",
    headers=get_headers(session_token)
)

print_response(response, "LIST FILES")

### 7.3 Ask Questions About a File

In [ ]:
# Ask a question about the uploaded file
file_question_data = {
    "question": "What is this document about?"
}

response = requests.post(
    f"{BASE_URL}/api/files/{uploaded_file_id}/read",
    json=file_question_data,
    headers=get_headers(session_token)
)

print_response(response, "FILE ANALYSIS")

### 7.4 Delete a File

In [ ]:
# Delete an uploaded file
# WARNING: This will permanently delete the file!
# Uncomment to actually delete

# response = requests.delete(
#     f"{BASE_URL}/api/files/{uploaded_file_id}",
#     headers=get_headers(session_token)
# )
# print_response(response, "DELETE FILE")

print("File deletion code is commented out for safety")

In [ ]:
# Query a specific path in the JSON data
json_path_chat_data = {
    "message": "What is the name and email of the first user?",
    "file_id": json_file_id,
    "json_path": "users.0"  # Access first user in the array
}

response = requests.post(
    f"{BASE_URL}/api/chat/with-json",
    json=json_path_chat_data,
    headers=get_headers(session_token)
)

print_response(response, "CHAT WITH SPECIFIC JSON PATH")

# Another example - query metadata only
print("\n" + "="*60)
print("Querying metadata section only...")
print("="*60)

json_path_chat_data2 = {
    "message": "What information is in this metadata?",
    "file_id": json_file_id,
    "json_path": "metadata"
}

response = requests.post(
    f"{BASE_URL}/api/chat/with-json",
    json=json_path_chat_data2,
    headers=get_headers(session_token)
)

print_response(response, "CHAT WITH METADATA PATH")


#### 7.5.4 Query Specific JSON Path

In [ ]:
# Chat with LLM using a reference to uploaded JSON file
json_file_chat_data = {
    "message": "List all the cities where users are located",
    "file_id": json_file_id  # Reference the uploaded file
}

response = requests.post(
    f"{BASE_URL}/api/chat/with-json",
    json=json_file_chat_data,
    headers=get_headers(session_token)
)

print_response(response, "CHAT WITH JSON FILE REFERENCE")


#### 7.5.3 Chat with Uploaded JSON File

In [ ]:
# Chat with LLM using inline JSON data
json_chat_data = {
    "message": "What is the average age of users in this dataset?",
    "json_data": sample_json_data  # Pass JSON directly
}

response = requests.post(
    f"{BASE_URL}/api/chat/with-json",
    json=json_chat_data,
    headers=get_headers(session_token)
)

print_response(response, "CHAT WITH INLINE JSON")


#### 7.5.2 Chat with Inline JSON Data

In [ ]:
# Create a sample JSON file for testing
sample_json_data = {
    "users": [
        {"id": 1, "name": "Alice", "age": 28, "email": "alice@example.com", "city": "New York"},
        {"id": 2, "name": "Bob", "age": 35, "email": "bob@example.com", "city": "San Francisco"},
        {"id": 3, "name": "Charlie", "age": 42, "email": "charlie@example.com", "city": "Chicago"},
        {"id": 4, "name": "Diana", "age": 31, "email": "diana@example.com", "city": "Boston"}
    ],
    "metadata": {
        "version": "1.0",
        "created": "2025-10-17",
        "total_users": 4
    }
}

# Save to file
with open("sample_users.json", "w") as f:
    json.dump(sample_json_data, f, indent=2)

# Upload the JSON file
with open("sample_users.json", "rb") as f:
    files = {"file": ("sample_users.json", f, "application/json")}
    response = requests.post(
        f"{BASE_URL}/api/files/upload",
        files=files,
        headers={"Authorization": f"Bearer {session_token}"}
    )

print_response(response, "UPLOAD JSON FILE")

# Store the file_id for JSON analysis
if response.status_code == 201:
    json_file_id = response.json().get("file_id")
    print(f"JSON file ID: {json_file_id}")
    
    # Now analyze the JSON file
    print("\n" + "="*60)
    print("Analyzing JSON structure...")
    print("="*60)
    
    response = requests.post(
        f"{BASE_URL}/api/files/{json_file_id}/analyze-json",
        headers=get_headers(session_token)
    )
    
    print_response(response, "JSON ANALYSIS")


#### 7.5.1 Upload and Analyze JSON File

## 7.5 JSON File Processing

The system provides special handling for JSON files with three powerful capabilities:
1. **Automatic Enhancement** - JSON files are automatically formatted with statistics
2. **Comprehensive Analysis** - Detailed JSON structure and quality analysis
3. **Direct Chat Integration** - Query JSON data with the LLM

## 8. System Health & Configuration <a name="system"></a>

### 8.1 Get System Information

In [ ]:
# Get system information (admin only)
response = requests.get(
    f"{BASE_URL}/api/system/info",
    headers=get_headers(session_token)
)

print_response(response, "SYSTEM INFO")

### 8.2 List Available Models

---

## Tips & Notes

### Authentication
- All endpoints except `/health` and `/api/auth/login` require authentication
- Include the Bearer token in the `Authorization` header
- Sessions expire after 24 hours of inactivity

### Session Management
- Use `session_id` to continue conversations
- New sessions are created automatically if no `session_id` is provided
- Sessions are user-specific and isolated

### RAG (Knowledge Base)
- RAG requires JSON files to be loaded into the system
- Set `use_rag: true` in chat requests to enable context injection
- The system searches for relevant content and adds it to the conversation context

### File Uploads
- Supported formats: PDF, DOCX, TXT, MD, XLSX, XLS, CSV, JSON, PY, JS, HTML, XML, YML, YAML
- Files are stored per-user in isolated directories
- Use `/api/files/:file_id/read` to analyze uploaded files with AI

### JSON Processing
- **Automatic Enhancement**: JSON files are automatically formatted with statistics when uploaded
- **Comprehensive Analysis**: Use `/api/files/:file_id/analyze-json` for detailed structure, schema, and quality analysis
- **Direct Chat Integration**: Use `/api/chat/with-json` to query JSON data directly with the LLM
- **Path Queries**: Use dot notation (e.g., `users.0.name`, `metadata.version`) to query specific parts of JSON
- **Inline or File Reference**: Pass JSON data directly or reference uploaded files via `file_id`

### Web Search
- Web search integrates with DuckDuckGo, Bing, and Brave
- Keyword extraction uses LLM to optimize search queries
- Results are cached for 1 hour by default

### Error Handling
- Always check `response.status_code` before processing results
- Common status codes:
  - `200`: Success
  - `201`: Created
  - `400`: Bad Request
  - `401`: Unauthorized
  - `403`: Forbidden (admin required)
  - `404`: Not Found
  - `500`: Server Error

### Admin Operations
- User management endpoints require admin role
- Model configuration changes require admin role
- Use with caution as these affect all users

---

**API Documentation**: See `API_DOCUMENTATION.md` for complete details

**Base URL**: `http://10.252.38.241:8000`

**API Version**: v2.1.1

**Last Updated**: 2025-10-17

### 8.3 Get Current Model Configuration

In [ ]:
# Get current model configuration
response = requests.get(
    f"{BASE_URL}/api/config/model",
    headers=get_headers(session_token)
)

print_response(response, "MODEL CONFIGURATION")

### 8.4 Update Model Configuration (Admin Only)

In [ ]:
# Update model configuration (admin only)
# WARNING: This changes system settings!
# Uncomment to actually update

# config_update = {
#     "ollama": {
#         "temperature": 0.8,
#         "num_ctx": 4096
#     }
# }
# 
# response = requests.post(
#     f"{BASE_URL}/api/config/model",
#     json=config_update,
#     headers=get_headers(session_token)
# )
# print_response(response, "UPDATE MODEL CONFIG")

print("Model configuration update code is commented out for safety")

## 9. Complete Workflow Example

This section demonstrates a complete workflow using multiple APIs together.

In [ ]:
print("="*60)
print("COMPLETE WORKFLOW EXAMPLE")
print("="*60)

# Step 1: Start a new conversation
print("\n1. Starting a conversation about Samsung Fold 7...")
chat_data = {
    "message": "What information do you have about Samsung Galaxy Fold 7?",
    "use_rag": True
}
response = requests.post(
    f"{BASE_URL}/api/chat/messages",
    json=chat_data,
    headers=get_headers(session_token)
)
if response.status_code == 200:
    workflow_session_id = response.json()["session_id"]
    print(f"✓ Conversation started (Session: {workflow_session_id})")
    print(f"Response preview: {response.json()['message'][:100]}...")

# Step 2: Continue the conversation
print("\n2. Asking a follow-up question...")
follow_up = {
    "message": "Tell me more about the battery life",
    "session_id": workflow_session_id,
    "use_rag": True
}
response = requests.post(
    f"{BASE_URL}/api/chat/messages",
    json=follow_up,
    headers=get_headers(session_token)
)
if response.status_code == 200:
    print(f"✓ Follow-up answered")
    print(f"Response preview: {response.json()['message'][:100]}...")

# Step 3: View conversation history
print("\n3. Viewing conversation history...")
response = requests.get(
    f"{BASE_URL}/api/chat/sessions/{workflow_session_id}",
    headers=get_headers(session_token)
)
if response.status_code == 200:
    history = response.json()["history"]
    print(f"✓ Conversation has {len(history)} messages")
    for msg in history:
        print(f"  - {msg['role']}: {msg['content'][:50]}...")

# Step 4: Perform a web search for latest information
print("\n4. Searching for latest Fold 7 news...")
search_data = {
    "query": "Samsung Galaxy Fold 7 2025 news",
    "max_results": 3
}
response = requests.post(
    f"{BASE_URL}/api/search/web",
    json=search_data,
    headers=get_headers(session_token)
)
if response.status_code == 200:
    results = response.json().get("results", [])
    print(f"✓ Found {len(results)} search results")
    for i, result in enumerate(results[:3], 1):
        print(f"  {i}. {result.get('title', 'No title')[:60]}...")

# Step 5: Check RAG system stats
print("\n5. Checking knowledge base statistics...")
response = requests.get(
    f"{BASE_URL}/api/rag/stats",
    headers=get_headers(session_token)
)
if response.status_code == 200:
    stats = response.json()["stats"]
    print(f"✓ Knowledge base ready")
    print(f"  - Documents: {stats.get('document_count', 0)}")
    print(f"  - Chunks: {stats.get('total_chunks', 0)}")
    print(f"  - Status: {stats.get('status', 'unknown')}")

print("\n" + "="*60)
print("WORKFLOW COMPLETE")
print("="*60)

## 10. Logout

In [ ]:
# Logout and invalidate session token
response = requests.post(
    f"{BASE_URL}/api/auth/logout",
    headers=get_headers(session_token)
)

print_response(response, "LOGOUT")

if response.status_code == 200:
    session_token = None
    print("Session token cleared")

---

## Tips & Notes

### Authentication
- All endpoints except `/health` and `/api/auth/login` require authentication
- Include the Bearer token in the `Authorization` header
- Sessions expire after 24 hours of inactivity

### Session Management
- Use `session_id` to continue conversations
- New sessions are created automatically if no `session_id` is provided
- Sessions are user-specific and isolated

### RAG (Knowledge Base)
- RAG requires JSON files to be loaded into the system
- Set `use_rag: true` in chat requests to enable context injection
- The system searches for relevant content and adds it to the conversation context

### File Uploads
- Supported formats: PDF, DOCX, TXT, MD, XLSX, XLS, CSV, JSON, PY, JS, HTML, XML, YML, YAML
- Files are stored per-user in isolated directories
- Use `/api/files/:file_id/read` to analyze uploaded files with AI

### Web Search
- Web search integrates with DuckDuckGo, Bing, and Brave
- Keyword extraction uses LLM to optimize search queries
- Results are cached for 1 hour by default

### Error Handling
- Always check `response.status_code` before processing results
- Common status codes:
  - `200`: Success
  - `201`: Created
  - `400`: Bad Request
  - `401`: Unauthorized
  - `403`: Forbidden (admin required)
  - `404`: Not Found
  - `500`: Server Error

### Admin Operations
- User management endpoints require admin role
- Model configuration changes require admin role
- Use with caution as these affect all users

---

**API Documentation**: See `API_DOCUMENTATION.md` for complete details

**Base URL**: `http://10.252.38.241:8000`

**Last Updated**: 2025-10-17